In [1]:
import torch
import numpy as np

In [2]:
import torch
from torch import nn
import torchvision # For all your CV task
import matplotlib.pyplot as plt

In [3]:
DATA_ROOT_DIR = "FashionMnist"

In [4]:
train_data = torchvision.datasets.FashionMNIST(
    root = DATA_ROOT_DIR,
    train = True,
    download = True,
    transform = torchvision.transforms.ToTensor(),
)


test_data = torchvision.datasets.FashionMNIST(
    root = DATA_ROOT_DIR,
    train = False,
    download = True,
    transform = torchvision.transforms.ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting FashionMnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMnist/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting FashionMnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMnist/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting FashionMnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMnist/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting FashionMnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMnist/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
BATCH_SIZE = 64

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [6]:
type(train_data_loader)

torch.utils.data.dataloader.DataLoader

In [7]:
for X, y in test_data_loader:
  print(X.shape)
  print(y.shape, y.dtype, torch.unique(y))
  break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()

    self.flatten = nn.Flatten()

    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512), # just like dense layer
        nn.ReLU(), # activation funcions
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU(),
        
    )


  def forward(self, X): # forward prop
    X = self.flatten(X)
    logits = self.linear_relu_stack(X)
    return logits

In [10]:
model = NeuralNetwork().to(device)

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimzer):
  size = len(dataloader.dataset)

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # calculate error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimzer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"training loss: {loss} [{current}/{size}]")

In [13]:
def test(dataloader, model):
  size = len(dataloader.dataset)
  model.eval()

  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss = test_loss +  loss_fn(pred, y).item()
      correct = correct + (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss = test_loss/size
  correct = correct/size

  print(f"Test loss: {100*correct}, average loss: {test_loss}")

In [14]:
EPOCHS = 3

for epoch in range(EPOCHS):
  print(f"EPOCH: {epoch + 1}\n-------------")

  train(train_data_loader, model, loss_fn, optimizer)
  test(test_data_loader, model)

print("Done")

EPOCH: 1
-------------
training loss: 2.307382106781006 [0/60000]
training loss: 2.2908174991607666 [6400/60000]
training loss: 2.2800798416137695 [12800/60000]
training loss: 2.2773144245147705 [19200/60000]
training loss: 2.2484171390533447 [25600/60000]
training loss: 2.2666735649108887 [32000/60000]
training loss: 2.237257719039917 [38400/60000]
training loss: 2.2288119792938232 [44800/60000]
training loss: 2.2231662273406982 [51200/60000]
training loss: 2.2155723571777344 [57600/60000]
Test loss: 51.65, average loss: 0.03454945394992828
EPOCH: 2
-------------
training loss: 2.1998796463012695 [0/60000]
training loss: 2.1768596172332764 [6400/60000]
training loss: 2.15399169921875 [12800/60000]
training loss: 2.1787638664245605 [19200/60000]
training loss: 2.1010332107543945 [25600/60000]
training loss: 2.159003496170044 [32000/60000]
training loss: 2.1025617122650146 [38400/60000]
training loss: 2.084812641143799 [44800/60000]
training loss: 2.084280490875244 [51200/60000]
trainin

In [15]:
torch.save(model, "model.pth")

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]


model.eval()


X = torch.tensor(test_data[0][0]).to(device)
y = torch.tensor(test_data[0][1]).to(device)

with torch.no_grad():
  pred = model(X)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(predicted, actual)

Ankle boot Ankle boot


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
